# Production Deployment with Ray Serve

**Prerequisites**: Complete [06_hyperparameter_tuning](../06_hyperparameter_tuning/01_ray_tune.ipynb)

You've trained a great policy. Now let's deploy it!

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                     FROM TRAINING TO PRODUCTION                             │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  TRAINING                                PRODUCTION                        │
│  ────────                                ──────────                        │
│                                                                             │
│  ┌─────────────┐                         ┌─────────────┐                   │
│  │   Train     │                         │   Serve     │                   │
│  │   Loop      │  ────> checkpoint ────> │   API       │                   │
│  │             │                         │             │                   │
│  └─────────────┘                         └─────────────┘                   │
│        │                                        │                          │
│        v                                        v                          │
│   Maximize reward                         Minimize latency                 │
│   Explore safely                          Serve reliably                   │
│   Takes hours                             Responds in ms                   │
│                                                                             │
│  DIFFERENT GOALS:                                                          │
│  Training: Learn the best policy                                           │
│  Serving: Use the policy fast and reliably                                 │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

## Ray Serve: Scalable Model Serving

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                          RAY SERVE ARCHITECTURE                             │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│     Client Request                                                         │
│          │                                                                  │
│          v                                                                  │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                        HTTP PROXY                                   │  │
│   │                    (handles routing)                                │  │
│   └────────────────────────────┬────────────────────────────────────────┘  │
│                                │                                           │
│         ┌──────────────────────┼──────────────────────┐                   │
│         │                      │                      │                   │
│         v                      v                      v                   │
│  ┌─────────────┐       ┌─────────────┐       ┌─────────────┐             │
│  │  Replica 1  │       │  Replica 2  │       │  Replica 3  │             │
│  │             │       │             │       │             │             │
│  │ ┌─────────┐ │       │ ┌─────────┐ │       │ ┌─────────┐ │             │
│  │ │ Policy  │ │       │ │ Policy  │ │       │ │ Policy  │ │             │
│  │ │ (copy)  │ │       │ │ (copy)  │ │       │ │ (copy)  │ │             │
│  │ └─────────┘ │       │ └─────────┘ │       │ └─────────┘ │             │
│  └─────────────┘       └─────────────┘       └─────────────┘             │
│                                                                            │
│  KEY FEATURES:                                                            │
│  • Auto-scaling: Add/remove replicas based on load                        │
│  • Batching: Combine requests for GPU efficiency                          │
│  • A/B Testing: Route traffic to different model versions                 │
│  • Zero-downtime: Update models without service interruption              │
│                                                                            │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Suppress warnings
import warnings
import logging
warnings.filterwarnings("ignore")
logging.getLogger("ray").setLevel(logging.ERROR)

import ray
from ray import serve
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.algorithm import Algorithm
import gymnasium as gym
import numpy as np
import time
from typing import Dict

ray.init(
    num_cpus=4,
    object_store_memory=1 * 1024 * 1024 * 1024,
    ignore_reinit_error=True,
)
print(f"Ray initialized: {ray.cluster_resources()}")

---

## Step 1: Train and Save a Model

In [ ]:
# Train a quick model for demo
config = (
    PPOConfig()
    .environment("CartPole-v1")
    .framework("torch")
    .env_runners(num_env_runners=2)
    .training(
        lr=3e-4,
        train_batch_size=2000,
    )
)

algo = config.build_algo()

print("Training model...")
print("=" * 50)
for i in range(5):
    result = algo.train()
    reward = result["env_runners"]["episode_return_mean"]
    print(f"Iter {i+1}: Reward = {reward:.1f}")

# Save checkpoint
checkpoint_path = algo.save()
print(f"\nCheckpoint saved to: {checkpoint_path}")

---

## Step 2: Create a Serve Deployment

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         DEPLOYMENT ANATOMY                                  │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  @serve.deployment(                                                        │
│      num_replicas=2,           # How many copies to run                   │
│      ray_actor_options={       # Resources per replica                    │
│          "num_cpus": 1,                                                   │
│          "num_gpus": 0,                                                   │
│      },                                                                    │
│  )                                                                         │
│  class PolicyServer:                                                       │
│                                                                             │
│      def __init__(self, checkpoint_path):                                  │
│          # Load model once at startup                                      │
│          self.algo = Algorithm.from_checkpoint(checkpoint_path)           │
│                                                                             │
│      async def __call__(self, request):                                    │
│          # Handle inference request                                        │
│          obs = await request.json()                                        │
│          action = self.algo.compute_single_action(obs)                    │
│          return {"action": action}                                        │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
@serve.deployment(
    num_replicas=1,  # Start with 1 for demo
    ray_actor_options={"num_cpus": 1},
)
class PolicyServer:
    """Ray Serve deployment for RL policy inference."""
    
    def __init__(self, checkpoint_path: str):
        """Load the trained algorithm from checkpoint."""
        print(f"Loading checkpoint: {checkpoint_path}")
        self.algo = Algorithm.from_checkpoint(checkpoint_path)
        self.request_count = 0
        print("Policy loaded and ready!")
    
    async def __call__(self, request) -> Dict:
        """Handle inference request."""
        start_time = time.time()
        
        # Parse request
        data = await request.json()
        observation = np.array(data["observation"])
        
        # Get action from policy
        action = self.algo.compute_single_action(observation)
        
        # Track metrics
        latency_ms = (time.time() - start_time) * 1000
        self.request_count += 1
        
        return {
            "action": int(action),
            "latency_ms": round(latency_ms, 2),
        }
    
    def get_metrics(self) -> Dict:
        """Return server metrics."""
        return {"request_count": self.request_count}

print("PolicyServer deployment class defined")

In [ ]:
# Deploy the server
serve.start()

# Bind the checkpoint path and deploy
deployment = PolicyServer.bind(checkpoint_path)
handle = serve.run(deployment, name="rl-policy")

print("\nServer deployed!")
print("Endpoint: http://localhost:8000/")

---

## Step 3: Test the Deployment

In [ ]:
import requests

# Get a sample observation
env = gym.make("CartPole-v1")
obs, _ = env.reset()
env.close()

print("Testing deployed policy...")
print("=" * 50)
print(f"Observation: {obs}")

# Send request to server
response = requests.post(
    "http://localhost:8000/",
    json={"observation": obs.tolist()}
)

result = response.json()
print(f"\nResponse: {result}")
print(f"  Action: {result['action']}")
print(f"  Latency: {result['latency_ms']:.2f} ms")

In [ ]:
# Run a full episode through the server
print("\nRunning full episode via API...")
print("=" * 50)

env = gym.make("CartPole-v1")
obs, _ = env.reset()
total_reward = 0
steps = 0
total_latency = 0

while True:
    # Get action from server
    response = requests.post(
        "http://localhost:8000/",
        json={"observation": obs.tolist()}
    )
    result = response.json()
    action = result["action"]
    total_latency += result["latency_ms"]
    
    # Step environment
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    steps += 1
    
    if terminated or truncated:
        break

env.close()

print(f"Episode completed!")
print(f"  Steps: {steps}")
print(f"  Total reward: {total_reward}")
print(f"  Avg latency: {total_latency/steps:.2f} ms/request")

---

## A/B Testing: Deploy Multiple Versions

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                            A/B TESTING                                      │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  Route traffic between model versions to test safely:                       │
│                                                                             │
│     Client Request                                                          │
│          │                                                                  │
│          v                                                                  │
│   ┌─────────────────────────────────────────────────┐                       │
│   │                   ROUTER                        │                       │
│   │                                                 │                       │
│   │  if random() < 0.9:   # 90% of traffic          │                       │
│   │      use Policy A (production)                  │                       │
│   │  else:                # 10% of traffic          │                       │
│   │      use Policy B (canary)                      │                       │
│   │                                                 │                       │
│   └──────────────────┬──────────────────────────────┘                       │
│                      │                                                      │
│           ┌──────────┴──────────┐                                           │
│           │                     │                                           │
│           v                     v                                           │
│   ┌─────────────────┐   ┌─────────────────┐                                 │
│   │   Policy A      │   │   Policy B      │                                 │
│   │  (production)   │   │   (canary)      │                                 │
│   │                 │   │                 │                                 │
│   │   90% traffic   │   │   10% traffic   │                                 │
│   └─────────────────┘   └─────────────────┘                                 │
│                                                                             │
│  WHY A/B TEST?                                                              │
│  - Test new models safely with small traffic percentage                     │
│  - Compare performance metrics between versions                             │
│  - Roll back instantly if new model performs worse                          │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## Monitoring Your Deployment

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         WHAT TO MONITOR                                     │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  LATENCY                                                                    │
│  ───────                                                                    │
│  - p50, p95, p99 latencies                                                  │
│  - Alert if p99 > threshold                                                 │
│                                                                             │
│     Latency │                                                               │
│       (ms)  │  ___    ___    ___                                            │
│        100  │ /   \  /   \  /   \   <-- spikes need investigation           │
│         50  │/     \/     \/     \___                                       │
│             └─────────────────────────                                      │
│                    Time                                                     │
│                                                                             │
│  ────────────────────────────────────────────────────────────────────       │
│                                                                             │
│  THROUGHPUT                                                                 │
│  ──────────                                                                 │
│  - Requests per second                                                      │
│  - Scale replicas if throughput < capacity                                  │
│                                                                             │
│  ────────────────────────────────────────────────────────────────────       │
│                                                                             │
│  ERRORS                                                                     │
│  ──────                                                                     │
│  - Error rate (should be < 0.1%)                                            │
│  - Track by error type                                                      │
│                                                                             │
│  ────────────────────────────────────────────────────────────────────       │
│                                                                             │
│  BUSINESS METRICS (RL-specific)                                             │
│  ──────────────────────────────                                             │
│  - Average reward in production                                             │
│  - Distribution shift detection                                             │
│  - Action distribution (is it reasonable?)                                  │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from collections import deque
import statistics

class PolicyMonitor:
    """Simple monitoring for deployed policies."""
    
    def __init__(self, window_size: int = 100):
        self.latencies = deque(maxlen=window_size)
        self.actions = deque(maxlen=window_size)
        self.errors = deque(maxlen=window_size)
    
    def record(self, latency_ms: float, action: int, error: bool = False):
        """Record a request."""
        self.latencies.append(latency_ms)
        self.actions.append(action)
        self.errors.append(error)
    
    def get_stats(self) -> Dict:
        """Get monitoring statistics."""
        if not self.latencies:
            return {"status": "no_data"}
        
        sorted_latencies = sorted(self.latencies)
        n = len(sorted_latencies)
        
        # Action distribution
        action_counts = {}
        for a in self.actions:
            action_counts[a] = action_counts.get(a, 0) + 1
        
        return {
            "requests": n,
            "latency_p50_ms": sorted_latencies[n // 2],
            "latency_p95_ms": sorted_latencies[int(n * 0.95)],
            "latency_mean_ms": statistics.mean(self.latencies),
            "error_rate": sum(self.errors) / len(self.errors),
            "action_distribution": action_counts,
        }

# Test the monitor
monitor = PolicyMonitor()

# Simulate some requests
env = gym.make("CartPole-v1")
obs, _ = env.reset()

for _ in range(50):
    response = requests.post(
        "http://localhost:8000/",
        json={"observation": obs.tolist()}
    )
    result = response.json()
    monitor.record(result["latency_ms"], result["action"])
    
    obs, _, done, _, _ = env.step(result["action"])
    if done:
        obs, _ = env.reset()

env.close()

print("Monitoring Stats:")
print("=" * 50)
stats = monitor.get_stats()
print(f"  Requests: {stats['requests']}")
print(f"  Latency p50: {stats['latency_p50_ms']:.2f} ms")
print(f"  Latency p95: {stats['latency_p95_ms']:.2f} ms")
print(f"  Error rate: {stats['error_rate']:.2%}")
print(f"  Actions: {stats['action_distribution']}")

---

## Checkpoints and Model Registry

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                        MODEL VERSIONING                                     │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  Track model versions like code versions!                                  │
│                                                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐   │
│  │                       MODEL REGISTRY                                │   │
│  ├─────────────────────────────────────────────────────────────────────┤   │
│  │  Name          │ Version │ Status     │ Reward │ Created           │   │
│  ├─────────────────┼─────────┼────────────┼────────┼───────────────────┤   │
│  │  cartpole_ppo  │ v1      │ archived   │ 180    │ 2024-01-01        │   │
│  │  cartpole_ppo  │ v2      │ production │ 450    │ 2024-01-15  ★     │   │
│  │  cartpole_ppo  │ v3      │ staging    │ 480    │ 2024-02-01        │   │
│  └─────────────────┴─────────┴────────────┴────────┴───────────────────┘   │
│                                                                             │
│  WORKFLOW:                                                                 │
│  1. Train new model → Save checkpoint                                      │
│  2. Register as "staging"                                                  │
│  3. A/B test against production                                           │
│  4. If better, promote to "production"                                    │
│  5. Archive old version                                                    │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

## Key Takeaways

1. **Ray Serve** provides scalable model serving with auto-scaling

2. **Load from checkpoint** in deployment `__init__` for fast startup

3. **Monitor latency, throughput, and errors** in production

4. **A/B test** new models safely before full rollout

5. **Version your models** like you version your code

## What's Next?

```
┌──────────────────┐          ┌──────────────────┐
│  07 Production   │          │ 08 Best Practice │
│  (you are here)  │   ───>   │                  │
│                  │          │  Industry        │
│  - Ray Serve     │          │  patterns for    │
│  - Monitoring    │          │  real-world RL   │
│  - A/B testing   │          │                  │
└──────────────────┘          └──────────────────┘
```

In [ ]:
# Cleanup
serve.shutdown()
algo.stop()
ray.shutdown()
print("Cleanup complete!")